This workbook was built by Pacific Broadband and Digital Equity (https://pacificbroadband.org) to explore ookla-open-data speeds specific to the United States Pacific territories. The beginning of the workbook in particular borrows from the tutorials at https://github.com/teamookla/ookla-open-data. Parsing out the data from millions and millions of quarterly speed reports into small slices takes significant time, especially just to read the data into a geopandas dataframe. Once that is done, this workbook generates an output geojson file (the format of which could easily be switched) and a data summary to text. 

Special thanks to Tyrone Taitano and Government of Guam for their support of the work.

The following cell is for running on GCP managed-notebook. Recommend n1-highmem-4 or higher. Installs on other platforms may vary.

In [ ]:
pip install geopandas adjustText matplotlib

In [ ]:
%matplotlib inline

from datetime import datetime

import geopandas as gp
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from shapely.geometry import Point
from adjustText import adjust_text

In [ ]:
def quarter_start(year: int, q: int) -> datetime:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")

    month = [1, 4, 7, 10]
    return datetime(year, month[q - 1], 1)


def get_tile_url(service_type: str, year: int, q: int) -> str:
    dt = quarter_start(year, q)

    base_url = "https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance"
    url = f"{base_url}/type%3D{service_type}/year%3D{dt:%Y}/quarter%3D{q}/{dt:%Y-%m-%d}_performance_{service_type}_tiles.zip"
    return url

Adjust the year and quarter in the following cell. Note that `place` is not used dynamically to get the quadkeys (it could be) and thus the script needs to be adjusted for whatever place is selected. Place at the moment just affects filename

In [ ]:
place = 'american-samoa'
year = 2020
quarter = 1

In [ ]:
tile_url = get_tile_url("fixed", year, quarter)
tile_url

In [ ]:
tiles = gp.read_file(tile_url)

In [ ]:
tiles.head()

In [ ]:
tiles_size = tiles.size
tiles_size

Adjust quadkeys in the following cell.

Representative quadkeys from https://labs.mapbox.com/what-the-tile/

guam 13320330, 13320331

cnmi 1332031

am-samoa 200021301, 20002131

usvi 0323002311, 0323003200, 032300213033, 032300213122, 032300213123, 0323002131323, 0323002131332, 0323002131333, 0323003020222, 0323002133111, 03230021313302, 03230021313303, 03230021313312, 03230021313313

oahu 022211110, 022211111, 022211113, 022211112

hawaii 022033, 022122, 022211, 022300

bozeman-mt 0213202213, 0213202302


In [ ]:
tiles1 = tiles[tiles['quadkey'].str.startswith('200021301') | tiles['quadkey'].str.startswith('20002131')]

In [ ]:
tiles1.head()

In [ ]:
tiles1_size = tiles1.size
tiles1_size

In [ ]:
outfile = (f'{place}_ookla_{year}Q{quarter}.geojson')
outfile

In [ ]:
tiles1.to_file(outfile, driver='GeoJSON')

In [ ]:
download = (tiles1['avg_d_kbps'].mean()) / 1000
upload = (tiles1['avg_u_kbps'].mean()) / 1000
latency = (tiles1['avg_lat_ms'].mean())
tests = (tiles1['tests']).sum()
devices = (tiles1['devices'].sum())
print(f'\n{place} {year}Q{quarter} Stats...\n Download (mean Mbps) {download}\n Upload (mean Mbps) {upload}\n Latency (mean ms) {latency}\n Tests {tests} Devices {devices}\n')